In [2]:
#Notebook for model 7
# decided to do this one once a semi understanding of how to manipluate and extract data from the relatively smaller data sets first 
# Will aim to implement the knowledge from preivous notebooks here 
# Aimning to extrapolate all the info and create and replecate similar procedures 
# 

In [3]:
#Import statements
# refer to documentation(WorkFlow)
# to understand why each library is used and what purpose(s)

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from scipy.spatial.distance import pdist, squareform
from skbio.stats.distance import mantel
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

In [4]:
csvfilePath = '/Users/schoudhry/Desktop/Supplementary Tables S7 - S7.csv'

In [ ]:
#Load into the DataFrame